## Setup
Import the standard Python Libraries that are used in this lab.


In [1]:
import boto3
from time import sleep
import subprocess
import pandas as pd
import json
import time

Import sagemaker and get execution role for getting role ARN

In [2]:
import sagemaker
region = boto3.Session().region_name    
smclient = boto3.Session().client('sagemaker')
from sagemaker import get_execution_role

role_arn = get_execution_role()
print(role_arn)

#Make sure this role has the forecast permissions set to be able to use S3

arn:aws:iam::325928439752:role/service-role/AmazonSageMaker-ExecutionRole-20190917T212788


The last part of the setup process is to validate that your account can communicate with Amazon Forecast, the cell below does just that.

In [3]:
session = boto3.Session(region_name='us-east-1') 
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

## Data Prepraration

In [4]:
df = pd.read_csv("../data/COF_yearly_Revenue_Data.csv", dtype = object, names=['metric_name','timestamp','metric_value'])
df.head(3)

,metric_name,timestamp,metric_value
0,metric_name,timestamp,metric_value
1,Revenue,2018-12-31,28076000000
2,Revenue,2017-12-31,27237000000


Create the training set and validation set. Use the last years revenue as the validation set

In [5]:
# Select 1996 to 2017 in one data frame
df_1996_2017 = df[(df['timestamp'] >= '1995-12-31') & (df['timestamp'] <= '2017-12-31')]

# Select the year 2018 seprately for validation
df = pd.read_csv("../data/COF_yearly_Revenue_Data.csv", dtype = object, names=['metric_name','timestamp','metric_value'])
df_2018 = df[(df['timestamp'] >= '2018-12-31')]

In [6]:
df_1996_2017


,metric_name,timestamp,metric_value
2,Revenue,2017-12-31,27237000000
3,Revenue,2016-12-31,25816000000
4,Revenue,2015-12-31,23413000000
5,Revenue,2014-12-31,22290000000
6,Revenue,2013-12-31,22384000000
7,Revenue,2012-12-31,21396000000
8,Revenue,2011-12-31,16279000000
9,Revenue,2010-12-31,16171000000
10,Revenue,2009-12-31,12983267000
11,Revenue,2008-12-31,13892686000


In [7]:
df_2018

,metric_name,timestamp,metric_value
0,metric_name,timestamp,metric_value
1,Revenue,2018-12-31,28076000000


Now export them to CSV files and place them into your data folder.

In [8]:
df_1996_2017.to_csv("../data/cof-revenue-train.csv", header=False, index=False)
df_2018.to_csv("../data/cof-revenue-validation.csv", header=False, index=False)

Define the S3 bucket name where we will upload data where Amazon Forecast will pick up the data later

In [9]:
bucket_name = "sagemaker-capone-forecast-useast1"  # Rember to change this to the correct bucket name used for Capital One


Upload the data to S3

In [10]:
s3 = session.client('s3')
key="pv/cof-revenue-train.csv"
s3.upload_file(Filename="../data/cof-revenue-train.csv", Bucket=bucket_name, Key=key)

## Creating the Dataset Group and Dataset <a class="anchor" id="dataset"></a>

In Amazon Forecast , a dataset is a collection of file(s) which contain data that is relevant for a forecasting task. A dataset must conform to a schema provided by Amazon Forecast. 

More details about `Domain` and dataset type can be found on the [documentation](https://docs.aws.amazon.com/forecast/latest/dg/howitworks-domains-ds-types.html) . For this example, we are using [METRICS](https://docs.aws.amazon.com/forecast/latest/dg/metrics-domain.html) domain with 3 required attributes `metrics_name`, `timestamp` and `metrics_value`.


It is importan to also convey how Amazon Forecast can understand your time-series information. That the cell immediately below does that, the next one configures your variable names for the Project, DatasetGroup, and Dataset.

In [11]:
DATASET_FREQUENCY = "Y" 
TIMESTAMP_FORMAT = "yyyy-mm-dd"

In [12]:
project = 'cof_revenue_forecastdemo'
datasetName= project+'_ds'
datasetGroupName= project +'_dsg'
s3DataPath = "s3://"+bucket_name+"/"+key

### Create the Dataset Group

In [13]:
create_dataset_group_response = forecast.create_dataset_group(DatasetGroupName=datasetGroupName,
                                                              Domain="METRICS",
                                                             )
datasetGroupArn = create_dataset_group_response['DatasetGroupArn']

In [14]:
forecast.describe_dataset_group(DatasetGroupArn=datasetGroupArn)

{'DatasetGroupName': 'cof_revenue_forecastdemo_dsg',
 'DatasetGroupArn': 'arn:aws:forecast:us-east-1:325928439752:dataset-group/cof_revenue_forecastdemo_dsg',
 'DatasetArns': [],
 'Domain': 'METRICS',
 'Status': 'ACTIVE',
 'CreationTime': datetime.datetime(2019, 10, 8, 17, 42, 56, 952000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2019, 10, 8, 17, 42, 56, 952000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '34643cf0-73b8-47ee-825e-66f65c2ab29e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 08 Oct 2019 17:42:58 GMT',
   'x-amzn-requestid': '34643cf0-73b8-47ee-825e-66f65c2ab29e',
   'content-length': '280',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

### Create the Schema

In [15]:
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
schema ={
   "Attributes":[
      {
         "AttributeName":"metric_name",
         "AttributeType":"string"
      },
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"metric_value",
         "AttributeType":"float"
      }
   ]
}

### Create the Dataset

In [16]:
response=forecast.create_dataset(
                    Domain="METRICS",
                    DatasetType='TARGET_TIME_SERIES',
                    DatasetName=datasetName,
                    DataFrequency=DATASET_FREQUENCY, 
                    Schema = schema
)

In [17]:
datasetArn = response['DatasetArn']
forecast.describe_dataset(DatasetArn=datasetArn)

{'DatasetArn': 'arn:aws:forecast:us-east-1:325928439752:dataset/cof_revenue_forecastdemo_ds',
 'DatasetName': 'cof_revenue_forecastdemo_ds',
 'Domain': 'METRICS',
 'DatasetType': 'TARGET_TIME_SERIES',
 'DataFrequency': 'Y',
 'Schema': {'Attributes': [{'AttributeName': 'metric_name',
    'AttributeType': 'string'},
   {'AttributeName': 'timestamp', 'AttributeType': 'timestamp'},
   {'AttributeName': 'metric_value', 'AttributeType': 'float'}]},
 'EncryptionConfig': {},
 'Status': 'ACTIVE',
 'CreationTime': datetime.datetime(2019, 10, 8, 17, 43, 3, 406000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2019, 10, 8, 17, 43, 3, 406000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '61a8291f-90e8-4804-b4c4-0e92035b56ae',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 08 Oct 2019 17:43:03 GMT',
   'x-amzn-requestid': '61a8291f-90e8-4804-b4c4-0e92035b56ae',
   'content-length': '520',
   'connection': 'keep-alive

### Add Dataset to Dataset Group

In [18]:
forecast.update_dataset_group(DatasetGroupArn=datasetGroupArn, DatasetArns=[datasetArn])

{'ResponseMetadata': {'RequestId': '90e5310a-aab0-4ec1-ac10-3234f2687d3e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 08 Oct 2019 17:43:06 GMT',
   'x-amzn-requestid': '90e5310a-aab0-4ec1-ac10-3234f2687d3e',
   'content-length': '2',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

### Create Data Import Job


Now that Forecast knows how to understand the CSV we are providing, the next step is to import the data from S3 into Amazon Forecaast.

In [19]:
datasetImportJobName = 'EP_DSIMPORT_JOB_TARGET'
ds_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=datasetImportJobName,
                                                          DatasetArn=datasetArn,
                                                          DataSource= {
                                                              "S3Config" : {
                                                                 "Path":s3DataPath,
                                                                 "RoleArn": role_arn
                                                              } 
                                                          },
                                                          TimestampFormat=TIMESTAMP_FORMAT
                                                         )

In [20]:
ds_import_job_arn=ds_import_job_response['DatasetImportJobArn']
print(ds_import_job_arn)

arn:aws:forecast:us-east-1:325928439752:dataset-import-job/cof_revenue_forecastdemo_ds/EP_DSIMPORT_JOB_TARGET


Check the status of dataset, when the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on the data size. It can take 10 mins to be **ACTIVE**. This process will take 5 to 10 minutes.

In [21]:
while True:
    dataImportStatus = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn)['Status']
    print(dataImportStatus)
    if dataImportStatus != 'ACTIVE' and dataImportStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break

CREATE_PENDING
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
ACTIVE


In [22]:
forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn)

{'DatasetImportJobName': 'EP_DSIMPORT_JOB_TARGET',
 'DatasetImportJobArn': 'arn:aws:forecast:us-east-1:325928439752:dataset-import-job/cof_revenue_forecastdemo_ds/EP_DSIMPORT_JOB_TARGET',
 'DatasetArn': 'arn:aws:forecast:us-east-1:325928439752:dataset/cof_revenue_forecastdemo_ds',
 'TimestampFormat': 'yyyy-mm-dd',
 'DataSource': {'S3Config': {'Path': 's3://sagemaker-capone-forecast-useast1/pv/cof-revenue-train.csv',
   'RoleArn': 'arn:aws:iam::325928439752:role/service-role/AmazonSageMaker-ExecutionRole-20190917T212788'}},
 'FieldStatistics': {'date': {'Count': 22,
   'CountDistinct': 22,
   'CountNull': 0,
   'Min': '1996-12-31T00:00:00Z',
   'Max': '2017-12-31T00:00:00Z'},
  'item': {'Count': 22, 'CountDistinct': 1, 'CountNull': 0},
  'target': {'Count': 22,
   'CountDistinct': 22,
   'CountNull': 0,
   'CountNan': 0,
   'Min': '1.203046E9',
   'Max': '2.7237E10',
   'Avg': 12856397727.272728,
   'Stddev': 8210568656.541229}},
 'DataSize': 6.26780092716217e-07,
 'Status': 'ACTIVE',
 

In [23]:
print("DatasetArn: ")
print(datasetGroupArn)

DatasetArn: 
arn:aws:forecast:us-east-1:325928439752:dataset-group/cof_revenue_forecastdemo_dsg
